In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install boto3 botocore awscli --ignore-installed
# !pip install ninja

# !pip install torch==1.9.0
# !pip install torchvision==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 4.8 MB/s 
     |████████████████████████████████| 8.9 MB 39.3 MB/s 
     |████████████████████████████████| 3.9 MB 42.6 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 247 kB 68.8 MB/s 
     |████████████████████████████████| 138 kB 70.1 MB/s 
     |████████████████████████████████| 636 kB 63.6 MB/s 
     |████████████████████████████████| 548 kB 66.2 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [ ]:
%cd drive/My\Drive/LELSD/notebooks

import numpy as np 
import matplotlib.pyplot as plt
import datetime
import os

import torch

import sys

sys.path.append("../")
import models
from lelsd import LELSD

from utils.stylegan2_utils import StyleGAN2SampleGenerator
from utils.stylegan3_utils import StyleGAN3SampleGenerator

import cv2
from tqdm import tqdm

import pickle
from spaceUtils import *

[Errno 2] No such file or directory: 'drive/MyDrive/LELSD/notebooks'
/content/drive/MyDrive/LELSD/notebooks


In [ ]:
def load_pretrained_model(model_name, dataset_name):
    if model_name != "biggan":
        G = models.get_model(model_name,
                             f"../pretrained/{model_name}/{model2available_dataset[model_name][dataset_name]}")
    else:
        G = models.get_model(model_name, model2available_dataset[model_name][dataset_name])
    return G

def get_batch_data(sample_generator, seed, model_name, dataset_name, bs=10):
    batch_data = sample_generator.generate_batch(seed, return_image=True, return_style=True, batch_size=bs)
    return batch_data

In [ ]:
model2available_dataset = {
    "stylegan2": {
        "FFHQ": "ffhq.pkl",
    },
    "stylegan3": {
        "FFHQ": "stylegan3-r-ffhqu-1024x1024.pkl",
    }
}

In [ ]:
# GLOBALS

# we fix seed sequence so to have all same generated images 
seqSize = 100
seedSeq = np.arange(seqSize)

savePath = '../logs/'

model_name     = 'stylegan3' # modify to the correct stylegan model you wish to compute info for
dataset_name   = 'FFHQ'
truncation_psi = 0.7 #default

In [ ]:
G      = load_pretrained_model(model_name, dataset_name)
device = torch.device('cuda')
if model_name == 'stylegan2':
    SG = StyleGAN2SampleGenerator(G=G, device=device, truncation_psi=truncation_psi)
elif model_name == 'stylegan3':
    SG = StyleGAN3SampleGenerator(G=G, device=device, truncation_psi=truncation_psi)

### 1. Generate and save Style Channels and Image 

#### StyleGan2 YS codes generating

In [ ]:
# # save and generate specific seed relate ys/rgbys vectors FOR STYLE GAN 2
# YS    = []
# RGBYS = []
# for idx, seed in tqdm(enumerate(seedSeq)):
#     data   = get_batch_data(SG, seed, model_name, dataset_name, bs=1)
#     ys     = data['ys']
#     rgb_ys = data['rgb_ys']

#     YS.append(ys)
#     RGBYS.append(rgb_ys)

# with open('{}/ysCodes.pkl'.format(savePath), 'wb') as f:
#     pickle.dump(YS, f)

# with open('{}/rgbysCodes.pkl'.format(savePath), 'wb') as f:
#     pickle.dump(RGBYS, f)

0it [00:00, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

1it [01:47, 107.22s/it]

Done.


100it [01:54,  1.15s/it]


#### StyleGan3 YS codes generating

In [ ]:
# # save and generate specific seed relate ys vectors FOR STYLE GAN 3
# YS    = []
# for idx, seed in tqdm(enumerate(seedSeq)):
#     data   = get_batch_data(SG, seed, model_name, dataset_name, bs=1)
#     ys     = data['ys']

#     YS.append(ys)

# with open('{}/ysCodesS3R.pkl'.format(savePath), 'wb') as f:
#     pickle.dump(YS, f)

0it [00:00, ?it/s]

Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


100it [01:51,  1.12s/it]


### 2. Generate and save Gradient Map of generated image (one layer: default=4) 

#### StyleGan2 Gradient Maps generating

In [ ]:
# # load previously generated ys codes from STYLE GAN 2
# with open('{}/ysCodes.pkl'.format(savePath), 'rb') as f:
#     YS = pickle.load(f)
# with open('{}/rgbysCodes.pkl'.format(savePath), 'rb') as f:
#     RGBYS = pickle.load(f)

In [ ]:
# # Uncomment for STYLE GAN 2
# L = 4 # modify here which layer you wish to compute the channels jacobian of

# for batch in range(0,10): # batches of 10 to compute because 100 takes too much space at once
#     GRADS = []
#     S,T   = batch * 10, (batch+1) * 10
#     for i in tqdm(range(S,T)):
#         # instantiating inputs of ys Latent Space
#         ys     = YS[i]
#         rgb_ys = RGBYS[i]

#         # here we computed all jacobians for the specific generated image that you see below (ys,rgb_ys are specific to that image)
#         lfunc, inp   = func_generator(SG, ys, rgb_ys, layer=L) 
#         grads        = jacobian(lfunc, inp)
#         GRADS.append(grads)

#     with open('{}/gradMapsS2/gradMapsB{}L{}.pkl'.format(savePath,batch,L), 'wb') as f:
#         pickle.dump(GRADS, f)

100%|██████████| 10/10 [31:13<00:00, 187.30s/it]


#### StyleGan3 Gradient Maps generating

In [ ]:
# # load previously generated ys codes from STYLE GAN 3 (by default R)
with open('{}/ysCodesS3R.pkl'.format(savePath), 'rb') as f:
    YS = pickle.load(f)

In [ ]:
# Uncomment for STYLE GAN 3
L = 4 # modify here which layer you wish to compute the channels jacobian of

for batch in range(0,10): # batches of 10 to compute because 100 takes too much space at once
    GRADS = []
    S,T   = batch * 10, (batch+1) * 10
    for i in tqdm(range(S,T)):
        # instantiating inputs of ys Latent Space
        ys     = YS[i]

        # here we computed all jacobians for the specific generated image that you see below (ys,rgb_ys are specific to that image)
        lfunc, inp   = func_generator2(SG, ys, layer=L) 
        grads        = jacobian(lfunc, inp)
        GRADS.append(grads)

    with open('{}/gradMapsS3R/gradMapsS3RB{}L{}.pkl'.format(savePath,batch,L), 'wb') as f:
        pickle.dump(GRADS, f)

 50%|█████     | 5/10 [1:00:15<1:00:16, 723.27s/it]

### 3. Semantic Masks Generating

In [ ]:
from utils.segmentation_utils import FaceSegmentation
face_bisenet      = models.get_model("face_bisenet", "../pretrained/face_bisenet/model.pth")
face_segmentation = FaceSegmentation(face_bisenet=face_bisenet, device=device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
indexMap = {0:0, 1:1, 2:2, 3:2, 4:3, 5:3, 6:4, 7:5, 8:5, 9:6, 10:7, 11:8, 12:8, 13:8, 14:9, 15:10, 16:11, 17:12, 18:13  }
indexMapping = lambda x: indexMap[x]

#### Mask generating for StyleGAN 2

In [ ]:
# load previously generated ys codes from STYLE GAN 2
with open('{}/ysCodes.pkl'.format(savePath), 'rb') as f:
    YS = pickle.load(f)
with open('{}/rgbysCodes.pkl'.format(savePath), 'rb') as f:
    RGBYS = pickle.load(f)

MASKS = []
for k in tqdm(range(len(YS))):
    img  = SG.generate_batch_from_ys((YS[k], RGBYS[k]), return_image=True)['image'][0]  
    mask = face_segmentation.predict(img)[0,0].cpu()    
    # mask = face_segmentation.predict(img)[0,0].cpu().apply_(indexMapping)
    MASKS.append(mask)

with open('{}/maskS2/maskS2.pkl'.format(savePath), 'wb') as f:
    pickle.dump(MASKS, f)

100%|██████████| 100/100 [00:09<00:00, 10.42it/s]


#### Mask generating for StyleGAN 3

In [ ]:
# load previously generated ys codes from STYLE GAN 3 (by default R)
with open('{}/ysCodesS3R.pkl'.format(savePath), 'rb') as f:
    YS = pickle.load(f)

MASKS = []
for k in tqdm(range(len(YS))):
    img  = SG.generate_batch_from_ys(YS[k], return_image=True)['image'][0]  
    mask = face_segmentation.predict(img)[0,0].cpu().apply_(indexMapping)
    MASKS.append(mask)

with open('{}/maskS3R/maskS3Rmerged.pkl'.format(savePath), 'wb') as f:
    pickle.dump(MASKS, f)    